In [1]:
import pandas as pd

### ground truth

In [2]:
# the ground truth values 
real_values = {
    'date': ['1/2024', '2/2024', '3/2024', '4/2024', '5/2024', '6/2024', '7/2024', '8/2024', '9/2024', '10/2024', '11/2024', '12/2024'],
    'real_consumo': [4885.10, 4880.14, 5391.35, 5925.34, 5295.28, 4996.13, 5256.78, 5069.64, 4694.26, 5215.3, 5273.88, 5501.23]
}

real_values = pd.DataFrame(real_values)
real_values = real_values.set_index('date')
real_values


,real_consumo
date,
1/2024,4885.10
2/2024,4880.14
3/2024,5391.35
4/2024,5925.34
5/2024,5295.28
6/2024,4996.13
7/2024,5256.78
8/2024,5069.64
9/2024,4694.26


# Xgboost

### Xgboost with energy prediction

In [3]:
prediction_data_path = 'results/output_data/xgboost_with_energy.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data

,datetime,predicted_consumo
0,2024-01-01,5.965704
1,2024-01-02,5.853434
2,2024-01-03,5.578959
3,2024-01-04,5.149488
4,2024-01-05,5.155605
...,...,...
361,2024-12-27,5.780270
362,2024-12-28,5.700403
363,2024-12-29,5.885840
364,2024-12-30,5.737703


In [4]:
# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

prediction_data

,predicted_consumo
date,
1/2024,4259.304041
2/2024,4534.367935
3/2024,4601.712974
4/2024,5617.293936
5/2024,5341.736280
6/2024,4121.975470
7/2024,4409.104291
8/2024,4827.393362
9/2024,4455.745277


In [5]:
# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

In [6]:
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4259.304041,625.795959,12.810300
2/2024,4880.14,4534.367935,345.772065,7.085290
3/2024,5391.35,4601.712974,789.637026,14.646369
4/2024,5925.34,5617.293936,308.046064,5.198791
5/2024,5295.28,5341.736280,-46.456280,-0.877315
6/2024,4996.13,4121.975470,874.154530,17.496633
7/2024,5256.78,4409.104291,847.675709,16.125379
8/2024,5069.64,4827.393362,242.246638,4.778379
9/2024,4694.26,4455.745277,238.514723,5.080987


In [7]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"std[erro] = {diff['Erro %'].std()} %")

E[erro] = 7.653871911201868 %
std[erro] = 6.91451821228947 %


### Xgboost without energy

In [8]:
prediction_data_path = 'results/output_data/xgboost_without_energy.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data

,datetime,predicted_consumo
0,2024-01-01,6.314996
1,2024-01-02,4.972981
2,2024-01-03,5.348199
3,2024-01-04,5.503366
4,2024-01-05,5.614134
...,...,...
361,2024-12-27,6.544522
362,2024-12-28,6.546956
363,2024-12-29,6.425216
364,2024-12-30,6.333966


In [9]:
# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

prediction_data

,predicted_consumo
date,
1/2024,4326.937819
2/2024,4469.639330
3/2024,4672.173120
4/2024,5580.077047
5/2024,5086.851252
6/2024,4392.707911
7/2024,4610.598091
8/2024,4526.254973
9/2024,4706.125030


In [10]:
# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

In [11]:
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4326.937819,558.162181,11.425809
2/2024,4880.14,4469.639330,410.500670,8.411658
3/2024,5391.35,4672.173120,719.176880,13.339458
4/2024,5925.34,5580.077047,345.262953,5.826888
5/2024,5295.28,5086.851252,208.428748,3.936123
6/2024,4996.13,4392.707911,603.422089,12.077790
7/2024,5256.78,4610.598091,646.181909,12.292352
8/2024,5069.64,4526.254973,543.385027,10.718414
9/2024,4694.26,4706.125030,-11.865030,-0.252756


In [12]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"sigma[erro] = {diff['Erro %'].std()} %")

E[erro] = 7.639081857359595 %
sigma[erro] = 5.609223050247394 %


### Xgboost shuffle True

In [39]:
prediction_data_path = 'results/output_data/xgboost_shuffle_v2.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data

,datetime,predicted_consumo
0,2024-01-01,6.176508
1,2024-01-02,4.996140
2,2024-01-03,5.290127
3,2024-01-04,5.372121
4,2024-01-05,5.351738
...,...,...
361,2024-12-27,6.658224
362,2024-12-28,6.836585
363,2024-12-29,7.111549
364,2024-12-30,7.146132


In [40]:
# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

prediction_data

,predicted_consumo
date,
1/2024,4610.914322
2/2024,4950.966967
3/2024,4996.607172
4/2024,5734.421263
5/2024,5380.678195
6/2024,4746.026378
7/2024,4922.102263
8/2024,4899.083959
9/2024,5067.481243


In [41]:
# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

In [42]:
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4610.914322,274.185678,5.612693
2/2024,4880.14,4950.966967,-70.826967,-1.451331
3/2024,5391.35,4996.607172,394.742828,7.321781
4/2024,5925.34,5734.421263,190.918737,3.222072
5/2024,5295.28,5380.678195,-85.398195,-1.612723
6/2024,4996.13,4746.026378,250.103622,5.005947
7/2024,5256.78,4922.102263,334.677737,6.366592
8/2024,5069.64,4899.083959,170.556041,3.364263
9/2024,4694.26,5067.481243,-373.221243,-7.950587


In [43]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"sigma[erro] = {diff['Erro %'].std()} %")

E[erro] = 1.8541535807312466 %
sigma[erro] = 5.030402135534392 %


#### Xgboost v3

In [48]:
prediction_data_path = 'results/output_data/xgboost_shuffle_v3.csv'
prediction_data = pd.read_csv(prediction_data_path)

# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4541.011018,344.088982,7.043643
2/2024,4880.14,4945.327169,-65.187169,-1.335764
3/2024,5391.35,5039.515020,351.834980,6.525916
4/2024,5925.34,5904.605702,20.734298,0.349926
5/2024,5295.28,5500.112338,-204.832338,-3.868206
6/2024,4996.13,4634.100024,362.029976,7.246208
7/2024,5256.78,4682.993868,573.786132,10.915164
8/2024,5069.64,4822.819330,246.820670,4.868603
9/2024,4694.26,5072.176730,-377.916730,-8.050614


In [49]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"sigma[erro] = {diff['Erro %'].std()} %")

E[erro] = 1.7770916371617622 %
sigma[erro] = 5.979317035726353 %


#### Xgboost v4

In [50]:
prediction_data_path = 'results/output_data/xgboost_shuffle_v4.csv'
prediction_data = pd.read_csv(prediction_data_path)

# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4591.130542,293.969458,6.017675
2/2024,4880.14,4990.735920,-110.595920,-2.266245
3/2024,5391.35,4936.707667,454.642333,8.432811
4/2024,5925.34,5739.046810,186.293190,3.144008
5/2024,5295.28,5235.855847,59.424153,1.122210
6/2024,4996.13,4609.923415,386.206585,7.730115
7/2024,5256.78,4730.686738,526.093262,10.007900
8/2024,5069.64,4771.092768,298.547232,5.888924
9/2024,4694.26,5144.833673,-450.573673,-9.598396


In [51]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"sigma[erro] = {diff['Erro %'].std()} %")

E[erro] = 2.573360614909053 %
sigma[erro] = 6.015089445576965 %


# Random Forest

## RF raposo com shuffle

In [18]:
prediction_data_path = 'results/output_data/Random_Forest_shuffle_True.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data

,datetime,predicted_consumo
0,2024-01-01,6.321083
1,2024-01-02,4.710533
2,2024-01-03,6.340784
3,2024-01-04,6.677676
4,2024-01-05,6.609860
...,...,...
361,2024-12-27,6.903014
362,2024-12-28,7.021107
363,2024-12-29,7.013029
364,2024-12-30,7.057451


In [19]:
# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

,predicted_consumo
date,
1/2024,4804.166352
2/2024,4924.504920
3/2024,5155.814535
4/2024,5831.911336
5/2024,5289.657295
6/2024,4674.052633
7/2024,4802.585968
8/2024,4842.705201
9/2024,5065.094750


In [20]:
# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

In [21]:
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4804.166352,80.933648,1.656745
2/2024,4880.14,4924.504920,-44.364920,-0.909091
3/2024,5391.35,5155.814535,235.535465,4.368766
4/2024,5925.34,5831.911336,93.428664,1.576765
5/2024,5295.28,5289.657295,5.622705,0.106183
6/2024,4996.13,4674.052633,322.077367,6.446537
7/2024,5256.78,4802.585968,454.194032,8.640157
8/2024,5069.64,4842.705201,226.934799,4.476349
9/2024,4694.26,5065.094750,-370.834750,-7.899749


In [22]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"sigma[erro] = {diff['Erro %'].std()} %")

E[erro] = 2.0389125766096328 %
sigma[erro] = 5.16092846232688 %


## RF raposo sem shuffle

In [23]:
prediction_data_path = 'results/output_data/Random_Forest_shuffle_False.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data

,datetime,predicted_consumo
0,2024-01-01,6.169879
1,2024-01-02,4.693347
2,2024-01-03,5.967931
3,2024-01-04,6.151734
4,2024-01-05,6.275802
...,...,...
361,2024-12-27,6.218974
362,2024-12-28,6.181078
363,2024-12-29,6.208962
364,2024-12-30,6.208202


In [24]:
# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

prediction_data

,predicted_consumo
date,
1/2024,4426.395957
2/2024,4487.261163
3/2024,4740.308226
4/2024,5654.582458
5/2024,5056.636438
6/2024,4306.554237
7/2024,4382.697699
8/2024,4449.504156
9/2024,4370.097183


In [25]:
# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

In [26]:
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4426.395957,458.704043,9.389860
2/2024,4880.14,4487.261163,392.878837,8.050565
3/2024,5391.35,4740.308226,651.041774,12.075673
4/2024,5925.34,5654.582458,270.757542,4.569485
5/2024,5295.28,5056.636438,238.643562,4.506722
6/2024,4996.13,4306.554237,689.575763,13.802198
7/2024,5256.78,4382.697699,874.082301,16.627713
8/2024,5069.64,4449.504156,620.135844,12.232345
9/2024,4694.26,4370.097183,324.162817,6.905515


In [27]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"sigma[erro] = {diff['Erro %'].std()} %")

E[erro] = 8.965682421213748 %
sigma[erro] = 5.383156445675574 %


## RF melhor que encontrei (sem shuffle)

In [28]:
prediction_data_path = 'results/output_data/best_Random_Forest_shuffle_False.csv'
prediction_data = pd.read_csv(prediction_data_path)
prediction_data

,datetime,predicted_consumo
0,2024-01-01,6.178144
1,2024-01-02,4.691731
2,2024-01-03,6.003979
3,2024-01-04,6.247457
4,2024-01-05,6.319898
...,...,...
361,2024-12-27,6.232412
362,2024-12-28,6.172702
363,2024-12-29,6.214657
364,2024-12-30,6.219020


In [29]:
# we will transform all the predicted values in MWh, so multiply by 24
prediction_data['predicted_consumo'] = prediction_data['predicted_consumo'] * 24

# Now, we want to group by month and sum the predicted energy
prediction_data['month'] = pd.to_datetime(prediction_data['datetime']).dt.month
prediction_data['year'] = pd.to_datetime(prediction_data['datetime']).dt.year
prediction_data = prediction_data.drop(columns=['datetime'])
prediction_data = prediction_data.groupby(['month', 'year']).sum()

# Only for beauty we'll join the month and year columns
prediction_data['date'] = prediction_data.index
prediction_data['date'] = prediction_data['date'].apply(lambda x: f'{x[0]}/{x[1]}')
prediction_data = prediction_data.reset_index()
prediction_data = prediction_data.drop(columns=['month', 'year'])
prediction_data = prediction_data.set_index('date')

prediction_data

,predicted_consumo
date,
1/2024,4450.469210
2/2024,4491.196632
3/2024,4766.603645
4/2024,5656.363185
5/2024,5071.906470
6/2024,4348.540024
7/2024,4439.331310
8/2024,4493.236735
9/2024,4381.525229


In [30]:
# Now we will make a dataframe that contains the difference between the real values and the predicted values
diff = real_values 
diff['predicted_consumo'] = prediction_data['predicted_consumo']
diff['Erro'] = diff['real_consumo'] - diff['predicted_consumo']
diff['Erro %'] = diff['Erro'] / diff['real_consumo'] * 100

In [31]:
diff

,real_consumo,predicted_consumo,Erro,Erro %
date,,,,
1/2024,4885.10,4450.469210,434.630790,8.897070
2/2024,4880.14,4491.196632,388.943368,7.969922
3/2024,5391.35,4766.603645,624.746355,11.587939
4/2024,5925.34,5656.363185,268.976815,4.539433
5/2024,5295.28,5071.906470,223.373530,4.218352
6/2024,4996.13,4348.540024,647.589976,12.961832
7/2024,5256.78,4439.331310,817.448690,15.550369
8/2024,5069.64,4493.236735,576.403265,11.369708
9/2024,4694.26,4381.525229,312.734771,6.662068


In [32]:
print(f"E[erro] = {diff['Erro %'].mean()} %")
print(f"sigma[erro] = {diff['Erro %'].std()} %")

E[erro] = 8.574133383193535 %
sigma[erro] = 5.127840454211631 %
